# Reshaping and pivot tables


There are three different ways to reshape an given DataFrame (or Series), and these are

- `pivot()`
- `stack()` `unstack()`
- `melt()`

These functions helps us to transitioned between different shapes forward and
backwards.

In [2]:
import pandas as pd
import numpy as np

np.random.seed(0)

In [19]:
## Handy functions
from IPython.display import display_html, display

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

def display_several(*args):
    for df in args:
        display(df)

## Wide and Long format

Suppose you have a variable `a` that depends on two parameters `i` and `j`. 
There are two equivalent ways to represent it as a table:

- **wide** format is more appropiate for dense data
- **large** format is more appropiate for sparse data, when some values are
zero or missing, and you can omit some rows.

<img src="./assets/imgs/wide_and_large.webp" width="500"/>

## `pivot()`

Pivot helps us to transition from a **long format** to a **wide format**. You 
need to provide the following

`df.pivot(index, columns, values)`

- `index` will be the column from the long format that will be transformed into the
index in the wide format.
- `columns` will be the column from the long format that will be transformed into
the columns in the wide format.
- `values` will be the column from the long format that will be the values in 
cells of the wide format.

<img src="./assets/imgs/reshaping_pivot.png" width="500"/>

Things to take into account when pivoting:

1. **values ommitted**: If `values` is ommited and there are several values columns in the long format, then there will be a multi-index in the columns indicating the respective value column.
2. **index ommitted**: If `index` is ommited, the index from the large format is used.
3. **missing values**: If there are missing values for the index/column pair, they will be
represented by NaN.

**NOTE:** `pivot()` will error with a ValueError: Index contains duplicate 
entries, **cannot reshape if the index/column pair is not unique**. In this 
case, consider using `pivot_table()` which is a generalization of pivot that can 
handle duplicate values for one index/column pair.

In [3]:
data = {
    "client" : ["John", "John", "Silvia", "Silvia"],
    "product" : ["bananas", "oranges", "bananas", "oranges"],
    "quantity" : [5, 3, 4, 2],
    "price" : [1.5, 3, 2.5, 4]
}

df = pd.DataFrame(data)
df

,client,product,quantity,price
0,John,bananas,5,1.5
1,John,oranges,3,3.0
2,Silvia,bananas,4,2.5
3,Silvia,oranges,2,4.0


In [5]:
# NOTE: client and product will be the index and columns respectively.
# And the values will be `quantity` discarting the column `price`.
result = df.pivot(index='client', columns='product', values='quantity')
display_side_by_side(df, result)

client 
 product 
 quantity 
 price 
 
 
 
 
 0 
 John 
 bananas 
 5 
 1.5 
 
 
 1 
 John 
 oranges 
 3 
 3.0 
 
 
 2 
 Silvia 
 bananas 
 4 
 2.5 
 
 
 3 
 Silvia 
 oranges 
 2 
 4.0 
 
 
 
 
 
 product 
 bananas 
 oranges 
 
 
 client 
 
 
 
 
 
 
 John 
 5 
 3 
 
 
 Silvia 
 4 
 2

In [6]:
# 1. values ommited
result = df.pivot(index='client', columns='product')
display_side_by_side(df, result)

client 
 product 
 quantity 
 price 
 
 
 
 
 0 
 John 
 bananas 
 5 
 1.5 
 
 
 1 
 John 
 oranges 
 3 
 3.0 
 
 
 2 
 Silvia 
 bananas 
 4 
 2.5 
 
 
 3 
 Silvia 
 oranges 
 2 
 4.0 
 
 
 
 
 
 
 quantity 
 price 
 
 
 product 
 bananas 
 oranges 
 bananas 
 oranges 
 
 
 client 
 
 
 
 
 
 
 
 
 John 
 5 
 3 
 1.5 
 3.0 
 
 
 Silvia 
 4 
 2 
 2.5 
 4.0

In [7]:
# 2. index ommited
result = df.pivot(columns='product', values='quantity')
display_side_by_side(df, result)

client 
 product 
 quantity 
 price 
 
 
 
 
 0 
 John 
 bananas 
 5 
 1.5 
 
 
 1 
 John 
 oranges 
 3 
 3.0 
 
 
 2 
 Silvia 
 bananas 
 4 
 2.5 
 
 
 3 
 Silvia 
 oranges 
 2 
 4.0 
 
 
 
 
 
 product 
 bananas 
 oranges 
 
 
 
 
 0 
 5.0 
 NaN 
 
 
 1 
 NaN 
 3.0 
 
 
 2 
 4.0 
 NaN 
 
 
 3 
 NaN 
 2.0

In [11]:
# 3. missing values (missing Silvia/oranges pair)
result = df.iloc[:-1, :].pivot(index='client', columns='product', values='quantity')
display_side_by_side(df.iloc[:-1, :], result)

client 
 product 
 quantity 
 price 
 
 
 
 
 0 
 John 
 bananas 
 5 
 1.5 
 
 
 1 
 John 
 oranges 
 3 
 3.0 
 
 
 2 
 Silvia 
 bananas 
 4 
 2.5 
 
 
 
 
 
 product 
 bananas 
 oranges 
 
 
 client 
 
 
 
 
 
 
 John 
 5.0 
 3.0 
 
 
 Silvia 
 4.0 
 NaN

## `pivot_table()`

The practice of grouping values and then pivoting the results is so common that `groupby` and `pivot` have been bundled together into a dedicated function (and a corresponding DataFrame method) `pivot_table`.

- without the `columns` argument, it behaves similarly to `groupby`;
- when there are no duplicate rows to group by, it works just like `pivot`;
- otherwise, it does grouping and pivoting.

In [126]:
data = {
    "client" : ["John", "John", "Silvia", "Silvia"],
    "product" : ["bananas", "oranges", "bananas", "oranges"],
    "quantity" : [5, 3, 4, 2],
    "price" : [1.5, 3, 2.5, 4]
}

df = pd.DataFrame(data)
df

,client,product,quantity,price
0,John,bananas,5,1.5
1,John,oranges,3,3.0
2,Silvia,bananas,4,2.5
3,Silvia,oranges,2,4.0


In [128]:
df.groupby(['client','product'])['quantity']

## `stack()` and `unstack()`

Pandas doesn’t have `set_index` for columns. A common way of adding levels to columns is to `unstack` existing levels from the index:

- `stack()` transform the inner-most column level into a inner-most index.
- `unstack()` makes the inverse process.

<img src="./assets/imgs/stack_unstack.webp" width="500"/>

Things to take into account when `stacking`:

1. **Control the level to stack (or Unstack)**: you can also select the level that you want to stack or unstack, using a name `stack(<'column_level_name'>)` or the level position `stack(<n_position>)`.
2. **Stacking multiple levels**: You can also stack or unstack more than one level at the same time by passing a list of levels and process them individually.
3. **Ending with a Series**: you can edd up with a Series if there are not more levels to stack.
4. **Missing Values**: `stack` or `unstack` can result in missing values if subgroups do not have the same set of labels, then it will be filled with NaN. You can fill the values using `fill_value = <number>`, but only in `unstack`.


**Note:** the `stack()` and `unstack()` methods implicitly sort the index levels involved.

**Note:** The columns index must not contain duplicate values to be eligible for stacking (same applies to index when unstacking):

In [27]:
columns = pd.MultiIndex.from_product([["population", "area"],[2010,2020]],
                                     names=["features", "year"])

index = pd.MultiIndex.from_tuples(
    [
        ("Portland", "Maine"),
        ("Portland", "Oregon"),
        ("Springfield","Illinois"),
        ("Springfield", "Oregon")
    ],
    names=["city", "region"]
)

data = np.array([[66194, 583776, 116250, 59403], 
                  [68408, 652503, 114394, 61851], 
                  [21.31, 133.43, 59.48, 15.74], 
                  [21.54, 133.45, 61.14, 15.85]]).T

df = pd.DataFrame(data, index=index, columns=columns)
df

features             population              area        
year                       2010      2020    2010    2020
city        region                                       
Portland    Maine       66194.0   68408.0   21.31   21.54
            Oregon     583776.0  652503.0  133.43  133.45
Springfield Illinois   116250.0  114394.0   59.48   61.14
            Oregon      59403.0   61851.0   15.74   15.85

In [28]:
# year becames the last level of the index
result = df.stack()
display_side_by_side(df, result)

features 
 population 
 area 
 
 
 
 year 
 2010 
 2020 
 2010 
 2020 
 
 
 city 
 region 
 
 
 
 
 
 
 
 
 Portland 
 Maine 
 66194.0 
 68408.0 
 21.31 
 21.54 
 
 
 Oregon 
 583776.0 
 652503.0 
 133.43 
 133.45 
 
 
 Springfield 
 Illinois 
 116250.0 
 114394.0 
 59.48 
 61.14 
 
 
 Oregon 
 59403.0 
 61851.0 
 15.74 
 15.85 
 
 
 
 
 
 
 
 features 
 area 
 population 
 
 
 city 
 region 
 year 
 
 
 
 
 
 
 Portland 
 Maine 
 2010 
 21.31 
 66194.0 
 
 
 2020 
 21.54 
 68408.0 
 
 
 Oregon 
 2010 
 133.43 
 583776.0 
 
 
 2020 
 133.45 
 652503.0 
 
 
 Springfield 
 Illinois 
 2010 
 59.48 
 116250.0 
 
 
 2020 
 61.14 
 114394.0 
 
 
 Oregon 
 2010 
 15.74 
 59403.0 
 
 
 2020 
 15.85 
 61851.0

In [29]:
# 1. control the level to stack (or unstack)
# alternative: df.stack(0) indicating the column level
result = df.stack('features')
display_side_by_side(df, result)

features 
 population 
 area 
 
 
 
 year 
 2010 
 2020 
 2010 
 2020 
 
 
 city 
 region 
 
 
 
 
 
 
 
 
 Portland 
 Maine 
 66194.0 
 68408.0 
 21.31 
 21.54 
 
 
 Oregon 
 583776.0 
 652503.0 
 133.43 
 133.45 
 
 
 Springfield 
 Illinois 
 116250.0 
 114394.0 
 59.48 
 61.14 
 
 
 Oregon 
 59403.0 
 61851.0 
 15.74 
 15.85 
 
 
 
 
 
 
 
 year 
 2010 
 2020 
 
 
 city 
 region 
 features 
 
 
 
 
 
 
 Portland 
 Maine 
 area 
 21.31 
 21.54 
 
 
 population 
 66194.00 
 68408.00 
 
 
 Oregon 
 area 
 133.43 
 133.45 
 
 
 population 
 583776.00 
 652503.00 
 
 
 Springfield 
 Illinois 
 area 
 59.48 
 61.14 
 
 
 population 
 116250.00 
 114394.00 
 
 
 Oregon 
 area 
 15.74 
 15.85 
 
 
 population 
 59403.00 
 61851.00

In [20]:
# 2. Stack (or unstack) multiple level at once
result = df.stack(['year', 'features'])
display_several(df, result)

features             population              area        
year                       2010      2020    2010    2020
Portland    Maine       66194.0   68408.0   21.31   21.54
            Oregon     583776.0  652503.0  133.43  133.45
Springfield Illinois   116250.0  114394.0   59.48   61.14
            Oregon      59403.0   61851.0   15.74   15.85

                       year  features  
Portland     Maine     2010  area              21.31
                             population     66194.00
                       2020  area              21.54
                             population     68408.00
             Oregon    2010  area             133.43
                             population    583776.00
                       2020  area             133.45
                             population    652503.00
Springfield  Illinois  2010  area              59.48
                             population    116250.00
                       2020  area              61.14
                             population    114394.00
             Oregon    2010  area              15.74
                             population     59403.00
                       2020  area              15.85
                             population     61851.00
dtype: float64

In [23]:
# 3. Ending with a Series.
# NOTE: the previous example can also be done with two consecutive stacks
result = df.stack().stack()
display_several(df, result)

features             population              area        
year                       2010      2020    2010    2020
Portland    Maine       66194.0   68408.0   21.31   21.54
            Oregon     583776.0  652503.0  133.43  133.45
Springfield Illinois   116250.0  114394.0   59.48   61.14
            Oregon      59403.0   61851.0   15.74   15.85

                       year  features  
Portland     Maine     2010  area              21.31
                             population     66194.00
                       2020  area              21.54
                             population     68408.00
             Oregon    2010  area             133.43
                             population    583776.00
                       2020  area             133.45
                             population    652503.00
Springfield  Illinois  2010  area              59.48
                             population    116250.00
                       2020  area              61.14
                             population    114394.00
             Oregon    2010  area              15.74
                             population     59403.00
                       2020  area              15.85
                             population     61851.00
dtype: float64

In [31]:
# 4. Missing Values are filled with NaN

result = df.iloc[0:-1,:].unstack()
display_side_by_side(df.iloc[0:-1,:], result)


features 
 population 
 area 
 
 
 
 year 
 2010 
 2020 
 2010 
 2020 
 
 
 city 
 region 
 
 
 
 
 
 
 
 
 Portland 
 Maine 
 66194.0 
 68408.0 
 21.31 
 21.54 
 
 
 Oregon 
 583776.0 
 652503.0 
 133.43 
 133.45 
 
 
 Springfield 
 Illinois 
 116250.0 
 114394.0 
 59.48 
 61.14 
 
 
 
 
 
 features 
 population 
 area 
 
 
 year 
 2010 
 2020 
 2010 
 2020 
 
 
 region 
 Illinois 
 Maine 
 Oregon 
 Illinois 
 Maine 
 Oregon 
 Illinois 
 Maine 
 Oregon 
 Illinois 
 Maine 
 Oregon 
 
 
 city 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Portland 
 NaN 
 66194.0 
 583776.0 
 NaN 
 68408.0 
 652503.0 
 NaN 
 21.31 
 133.43 
 NaN 
 21.54 
 133.45 
 
 
 Springfield 
 116250.0 
 NaN 
 NaN 
 114394.0 
 NaN 
 NaN 
 59.48 
 NaN 
 NaN 
 61.14 
 NaN 
 NaN

In [33]:
# You can fill the values using fill_value
# NOTE: stack doesn't have fill_value

result = df.iloc[0:-1,:].unstack(fill_value=1)
display_side_by_side(df.iloc[0:-1,:], result)

features 
 population 
 area 
 
 
 
 year 
 2010 
 2020 
 2010 
 2020 
 
 
 city 
 region 
 
 
 
 
 
 
 
 
 Portland 
 Maine 
 66194.0 
 68408.0 
 21.31 
 21.54 
 
 
 Oregon 
 583776.0 
 652503.0 
 133.43 
 133.45 
 
 
 Springfield 
 Illinois 
 116250.0 
 114394.0 
 59.48 
 61.14 
 
 
 
 
 
 features 
 population 
 area 
 
 
 year 
 2010 
 2020 
 2010 
 2020 
 
 
 region 
 Illinois 
 Maine 
 Oregon 
 Illinois 
 Maine 
 Oregon 
 Illinois 
 Maine 
 Oregon 
 Illinois 
 Maine 
 Oregon 
 
 
 city 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Portland 
 1.0 
 66194.0 
 583776.0 
 1.0 
 68408.0 
 652503.0 
 1.00 
 21.31 
 133.43 
 1.00 
 21.54 
 133.45 
 
 
 Springfield 
 116250.0 
 1.0 
 1.0 
 114394.0 
 1.0 
 1.0 
 59.48 
 1.00 
 1.00 
 61.14 
 1.00 
 1.00

### Use `stack()` to unpivot

You can use `stack()` together with `reset_index()` to unpivot a given wide 
format, but take into account that is not its main behavior.


In [121]:
data = {
    "client" : ["John", "John", "Silvia", "Silvia"],
    "product" : ["bananas", "oranges", "bananas", "oranges"],
    "quantity" : [5, 3, 4, 2],
    "price" : [1.5, 3, 2.5, 4]
}

df = pd.DataFrame(data)
df = df.pivot(index="client", columns="product", values="quantity")
df

product,bananas,oranges
client,,
John,5,3
Silvia,4,2


In [122]:
df.stack().reset_index()

,client,product,0
0,John,bananas,5
1,John,oranges,3
2,Silvia,bananas,4
3,Silvia,oranges,2


In [123]:
df = df.stack() 
df.reset_index(name = "quantity")

,client,product,quantity
0,John,bananas,5
1,John,oranges,3
2,Silvia,bananas,4
3,Silvia,oranges,2


## `melt()`

`melt()` practically takes a wide format and transform it in the long format, but we need **previously reset index** if we have a pivot table. It is because `melt()` works with columns.


`melt(id_vars, value_vars, var_name, value_name)`

- `id_vars` indicates the columns that are used as identifiers (which were transformed to index with pivot).
- `value_vars` indicates the columns to unpivot (which were transformed to columns with pivot). If not specified, uses all columns that are not set as `id_vars`.
- `var_name` sets the name of variable column (default = variable).
- `value_name` sets the name of value column (dafault = value).


**Note:** note the difference with a pure unpivot is that you can set different
number of `id_vars` which are **taken from the columns not the index**. You can also
set different `value_vars` to make a kind of filtering.

<img src="./assets/imgs/reshaping_melt.png" width="500"/>

**Note:** the `melt()` main behavior is to unpivot, in contrast
to `stack()`. Although `stack()` could be used to unpivot, it is not its main
behavior.

**Note:** `melt()` will ignore the index by default. Use `ignore_index = False`
to keep it.

In [125]:
data = { "bananas" : [5,3],
         "oranges" : [4, 2]}

index = pd.Index(["John", "Silvia"], name="client")

df = pd.DataFrame(data, index=index)
df

#Note this is the wide format version

,bananas,oranges
client,,
John,5,4
Silvia,3,2


In [105]:
# We need to reset the index because melt() works in columns
df = df.reset_index()
df

,client,bananas,oranges
0,John,5,4
1,Silvia,3,2


In [106]:
# We get the long format
df.melt(id_vars=["client"])

,client,variable,value
0,John,bananas,5
1,Silvia,bananas,3
2,John,oranges,4
3,Silvia,oranges,2


In [108]:
# setting name to the new columns
df.melt(id_vars=["client"], var_name="product", value_name="quantity")

,client,product,quantity
0,John,bananas,5
1,Silvia,bananas,3
2,John,oranges,4
3,Silvia,oranges,2


In [111]:
# we can also select which value vars to use to perform a kind of filtering
df.melt(id_vars=["client"], value_vars=["bananas"], var_name="product", value_name="quantity")

,client,product,quantity
0,John,bananas,5
1,Silvia,bananas,3


# Use Cases for reshaping

### Combining with stats and groupby

In [83]:
columns = pd.MultiIndex.from_product([["A", "B"], ["cat", "dog"]]
                                     , names=["letter", "animal"])

index = pd.MultiIndex.from_product([["bar", "baz", "foo", "qux"], ["one", "two"] ])

df = pd.DataFrame(
    data = np.random.randint(20, size=(8,4)),
    index= index,
    columns= columns
)
df

letter    A       B    
animal  cat dog cat dog
bar one  12   2   2   1
    two   5   8   4   0
baz one  11   2   5  16
    two   8   1  17  16
foo one   3   8   4  16
    two   3   7   3  18
qux one  17   1  18  17
    two   2   5   5  12

#### Countraintuive Explanation for axis using aggreagtion

It is countraintuive for me to get

`mean(axis = 0)`
`mean(axis = 1)`

It is because

1. axis = 0 indicates rows, but it will got one mean per each column
2. axis = 1 indicates cols, but it will got one mean per each row.

It is similar to any aggregation function.


In [84]:
# NOTE: you can get both results in diff ways
# NOTE: in group by the axis will indicate the way that mean() behaves too
# so it will also get the mean on axis = 1

result1 = df.groupby(level=1, axis=1).mean()
result2 = df.stack().mean(1).unstack()
display_side_by_side(result1, result2)

animal 
 cat 
 dog 
 
 
 
 
 bar 
 one 
 7.0 
 1.5 
 
 
 two 
 4.5 
 4.0 
 
 
 baz 
 one 
 8.0 
 9.0 
 
 
 two 
 12.5 
 8.5 
 
 
 foo 
 one 
 3.5 
 12.0 
 
 
 two 
 3.0 
 12.5 
 
 
 qux 
 one 
 17.5 
 9.0 
 
 
 two 
 3.5 
 8.5 
 
 
 
 
 
 
 animal 
 cat 
 dog 
 
 
 
 
 bar 
 one 
 7.0 
 1.5 
 
 
 two 
 4.5 
 4.0 
 
 
 baz 
 one 
 8.0 
 9.0 
 
 
 two 
 12.5 
 8.5 
 
 
 foo 
 one 
 3.5 
 12.0 
 
 
 two 
 3.0 
 12.5 
 
 
 qux 
 one 
 17.5 
 9.0 
 
 
 two 
 3.5 
 8.5

In [92]:
result1 = df.stack().groupby(level="animal").mean()
result2 = df.mean().unstack(0)
display_side_by_side(result1, result2)

letter,A,B
animal,,
cat,7.625,7.25
dog,4.250,12.00
letter,A,B
animal,,
cat,7.625,7.25
dog,4.250,12.00
